In [14]:
import json
import itertools
import operator
from transformers import AutoTokenizer, AutoModel
from seqlbtoolkit.embs import build_bert_token_embeddings
from seqlbtoolkit.text import split_overlength_bert_input_sequence


In [12]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
with open('a.json', 'r', encoding='utf-8') as f:
    text_seq = json.load(f)

In [6]:
encs = tokenizer(text_seq,
                 is_split_into_words=True,
                 add_special_tokens=True,
                 return_offsets_mapping=True)


In [28]:
max_seq_length = 512
tk_seq_list = [text_seq]
sent_lengths_list = [None]

split_tk_seq_list = list()
ori2split_ids_map = list()
n = 0

# update input sentences so that every sentence has BERT length < 510
for tk_seq, sent_lens in zip(tk_seq_list, sent_lengths_list):

    if sent_lens:
        ends = list(itertools.accumulate(sent_lens, operator.add))
        starts = [0] + ends[:-1]
        tk_seq_ = [tk_seq[s:e] for s, e in zip(starts, ends)]
    else:
        tk_seq_ = [tk_seq]

    tk_seqs = split_overlength_bert_input_sequence(tk_seq_, tokenizer, max_seq_length)
    n_splits = len(tk_seqs)
    split_tk_seq_list += tk_seqs

    ori2split_ids_map.append(list(range(n, n + n_splits)))
    n += n_splits


In [20]:
len(split_tk_seq_list[0])

367

In [21]:
len(text_seq)

366

In [23]:
len(tk_seq)

366

In [29]:
len(tk_seqs[0])

366